In [1]:
import tensorflow as tf   # http://blog.topspeedsnail.com/archives/10399
from sklearn.preprocessing import scale  # 使用scikit-learn进行数据预处理
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [2]:
from functools import partial

In [3]:
training_data = pd.read_csv("F:/UJIndoorLoc/trainingData.csv",engine='python',header = 0)
train_x = scale(np.asarray(training_data.iloc[:,0:520])) #归一化
train_y = np.asarray(training_data["BUILDINGID"].map(str) + training_data["FLOOR"].map(str))  
train_y = np.asarray(pd.get_dummies(train_y)) #进行one-hot  

# X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=0)
# Y_train=np.asarray(pd.get_dummies(Y_train)) #进行one-hot  
# Y_test=np.asarray(pd.get_dummies(Y_test)) #进行one-hot  

# train_x=X_train
# train_y=Y_train
# test_x=X_test
# test_y=Y_test

test_dataset = pd.read_csv("F:/UJIndoorLoc/validationData.csv",engine='python',header = 0)
test_x = scale(np.asarray(test_dataset.iloc[:,0:520]))
test_y = np.asarray(test_dataset["BUILDINGID"].map(str) + test_dataset["FLOOR"].map(str))
test_y = np.asarray(pd.get_dummies(test_y)) #进行one-hot  

D:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [4]:
X_train_tar, X_test_tar, Y_train_tar, Y_test_tar = train_test_split(test_x, test_y, test_size=0.3, random_state=0)

In [4]:
# train_x=np.concatenate((train_x[:8000],test_x[:800]), axis=0)
# train_y=np.concatenate((train_y[:8000],test_y[:800]), axis=0)
# X_train, X_test, Y_train, Y_test = train_test_split(train_x, train_y, test_size=0.3, random_state=0)
# test_x=test_x[801:]
# test_y=test_y[801:]

In [14]:
output = train_y.shape[1]
with tf.name_scope('input'):
    X = tf.placeholder(tf.float32, shape=[None, 520],name = 'x_input')  # 网络输入
    Y = tf.placeholder(tf.float32,[None, output],name = 'y_input') # 网络输出
# 定义神经网络
batch_norm_momentum = 0.6
training = tf.placeholder_with_default(False, shape=(), name='training')
my_batch_norm_layer = partial(tf.layers.batch_normalization,training=training,momentum=batch_norm_momentum)
# def neural_networks():
	# --------------------- Encoder -------------------- #
with tf.name_scope('encoder'):
    e_w_1 = tf.Variable(tf.truncated_normal([520, 256], stddev = 0.1),name='e_w_1')
    e_b_1 = tf.Variable(tf.constant(0.0, shape=[256]),name='e_b_1')
    e_w_2 = tf.Variable(tf.truncated_normal([256, 128], stddev = 0.1),name='e_w_2')
    e_b_2 = tf.Variable(tf.constant(0.0, shape=[128]),name='e_b_2')
    e_w_3 = tf.Variable(tf.truncated_normal([128, 64], stddev = 0.1),name='e_w_3')
    e_b_3 = tf.Variable(tf.constant(0.0, shape=[64]),name='e_b_3')
# --------------------- Decoder  ------------------- #
with tf.name_scope('decoder'):
    d_w_1 = tf.Variable(tf.truncated_normal([64, 128], stddev = 0.1),name='d_w_1')
    d_b_1 = tf.Variable(tf.constant(0.0, shape=[128]),name='d_b_1')
    d_w_2 = tf.Variable(tf.truncated_normal([128, 256], stddev = 0.1),name='d_w_2')
    d_b_2 = tf.Variable(tf.constant(0.0, shape=[256]),name='d_b_2')
    d_w_3 = tf.Variable(tf.truncated_normal([256, 520], stddev = 0.1),name='d_w_3')
    d_b_3 = tf.Variable(tf.constant(0.0, shape=[520]),name='d_b_3')
# --------------------- DNN  ------------------- #
with tf.name_scope('dnn'):
    w_1 = tf.Variable(tf.truncated_normal([64, 128], stddev = 0.1),name='w_1')
    b_1 = tf.Variable(tf.constant(0.0, shape=[128]),name='b_1')
    w_2 = tf.Variable(tf.truncated_normal([128, 128], stddev = 0.1),name='w_2')
    b_2 = tf.Variable(tf.constant(0.0, shape=[128]),name='b_2')
# --------------------- Softmax  ------------------- #
with tf.name_scope('output'):    
    w_3 = tf.Variable(tf.truncated_normal([128, output], stddev = 0.1),name='w_3')
    b_3 = tf.Variable(tf.constant(0.0, shape=[output]),name='b_3')
#########################################################
layer_1 = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(X,       e_w_1), e_b_1)))
layer_1 = tf.layers.dropout(layer_1, keep_prob)

layer_2 = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(layer_1, e_w_2), e_b_2)))
layer_2 = tf.layers.dropout(layer_2, keep_prob)

encoded = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(layer_2, e_w_3), e_b_3)))
encoded = tf.layers.dropout(encoded, keep_prob)

layer_4 = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(encoded, d_w_1), d_b_1)))
layer_4 = tf.layers.dropout(layer_4, keep_prob)

layer_5 = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(layer_4, d_w_2), d_b_2)))
decoded = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(layer_5, d_w_3), d_b_3)))

layer_7 = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(encoded, w_1),   b_1)))
layer_7 = tf.layers.dropout(layer_7, keep_prob)
layer_8 = tf.nn.tanh(my_batch_norm_layer(tf.add(tf.matmul(layer_7, w_2),   b_2)))
out = tf.nn.softmax(tf.add(tf.matmul( layer_8, w_3),   b_3))
# 	return (decoded, out)

In [11]:
# 训练神经网络
def train_neural_networks():
# 	decoded, predict_output = neural_networks()
	predict_output = out

	us_cost_function = tf.reduce_mean(tf.pow(X - decoded, 2))
	s_cost_function = -tf.reduce_sum(Y * tf.log(predict_output))
	#us_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(us_cost_function)
	us_optimizer = tf.train.AdamOptimizer(1e-4).minimize(us_cost_function)
	#s_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(s_cost_function)
	s_optimizer = tf.train.AdamOptimizer(1e-4).minimize(s_cost_function)
	correct_prediction = tf.equal(tf.argmax(predict_output, 1), tf.argmax(Y,1))
	accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
	training_epochs = 10
	batch_size = 20
	total_batches = training_data.shape[0]
    
	saver = tf.train.Saver()
    
	with tf.Session() as sess:      
		sess.run(tf.global_variables_initializer())   # 新的
		# ------------ Traiing Autoencoders - Unsupervised Learning ----------- #
		# autoencoder是一种非监督学习算法，他利用反向传播算法，让目标值等于输入值
		for epoch in range(training_epochs):
			epoch_costs = np.empty(0)
			for b in range(total_batches):
				offset = (b * batch_size) % (train_x.shape[0] - batch_size)
				batch_x = train_x[offset:(offset + batch_size), :]
				_, c = sess.run([us_optimizer, us_cost_function],feed_dict={X: batch_x,keep_prob: 0.5})
				epoch_costs = np.append(epoch_costs, c)
			print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs))
		print("------------------------------------------------------------------")
 
		# ---------------- Training NN - Supervised Learning ------------------ #
# 		for epoch in range(training_epochs):
# 			epoch_costs = np.empty(0)
# 			for b in range(total_batches):
# 				offset = (b * batch_size) % (train_x.shape[0] - batch_size)
# 				batch_x = train_x[offset:(offset + batch_size), :]
# 				batch_y = train_y[offset:(offset + batch_size), :]
# 				_, c = sess.run([s_optimizer, s_cost_function],feed_dict={X: batch_x, Y : batch_y})
# 				epoch_costs = np.append(epoch_costs,c)
# 			accuracy_in_train_set = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
# 			accuracy_in_test_set = sess.run(accuracy, feed_dict={X: test_x, Y: test_y})
# 			print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs)," Accuracy: ", accuracy_in_train_set, ' ', accuracy_in_test_set)

		# ---------------- Training NN - Supervised Learning ------------------ #
		for epoch in range(training_epochs):
			epoch_costs = np.empty(0)
			for b in range(total_batches):
				offset = (b * batch_size) % (train_x.shape[0] - batch_size)
				batch_x = train_x[offset:(offset + batch_size), :]
				batch_y = train_y[offset:(offset + batch_size), :]
				_, c = sess.run([s_optimizer, s_cost_function],feed_dict={X: batch_x, Y : batch_y,keep_prob: 0.5})
				epoch_costs = np.append(epoch_costs,c)
			accuracy_in_train_set = sess.run(accuracy, feed_dict={X: train_x, Y: train_y})
			accuracy_in_test_set = sess.run(accuracy, feed_dict={X: test_x, Y: test_y})
			print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs)," Accuracy: ", accuracy_in_train_set, ' ', accuracy_in_test_set)


		saver.save(sess, './cppp1/test-model',global_step=1000)
        
train_neural_networks()

Epoch:  0  Loss:  0.696978690206
Epoch:  1  Loss:  0.669638258673
Epoch:  2  Loss:  0.664176695099
Epoch:  3  Loss:  0.66096118152
Epoch:  4  Loss:  0.658741698858
Epoch:  5  Loss:  0.657143499316
Epoch:  6  Loss:  0.655902985316
Epoch:  7  Loss:  0.654889446622
Epoch:  8  Loss:  0.654100208088
Epoch:  9  Loss:  0.653418596451
------------------------------------------------------------------
Epoch:  0  Loss:  5.11026153704  Accuracy:  0.929378   0.660666
Epoch:  1  Loss:  1.40857149276  Accuracy:  0.949541   0.650765
Epoch:  2  Loss:  0.70688988134  Accuracy:  0.974169   0.667867
Epoch:  3  Loss:  0.36714353816  Accuracy:  0.990972   0.70297
Epoch:  4  Loss:  0.237315686504  Accuracy:  0.995235   0.693069
Epoch:  5  Loss:  0.202713684724  Accuracy:  0.99358   0.69667
Epoch:  6  Loss:  0.171761010828  Accuracy:  0.996589   0.704771
Epoch:  7  Loss:  0.160509846913  Accuracy:  0.996238   0.710171
Epoch:  8  Loss:  0.150868761484  Accuracy:  0.995887   0.693069
Epoch:  9  Loss:  0.146568

In [12]:
with tf.Session() as sess2:
#先加载图和参数变量
    saver_load = tf.train.import_meta_graph('./cppp1/test-model-1000.meta')
    saver_load.restore(sess2, tf.train.latest_checkpoint('./cppp1'))
    graph = tf.get_default_graph()
#     e_w_1 = graph.get_tensor_by_name("e_w_1:0")
#     e_w_2 = graph.get_tensor_by_name("e_w_2:0")
#     e_w_3 = graph.get_tensor_by_name("e_w_3:0")
#     d_w_1 = graph.get_tensor_by_name("d_w_1:0")
#     d_w_2 = graph.get_tensor_by_name("d_w_2:0")
#     d_w_3 = graph.get_tensor_by_name("d_w_3:0")
#     w_1 = graph.get_tensor_by_name("w1:0")
#     w_2 = graph.get_tensor_by_name("w2:0")
#     #w_3 = graph.get_tensor_by_name("w_3:0")
#     e_b_1 = graph.get_tensor_by_name("e_b_1")
#     e_b_2 = graph.get_tensor_by_name("e_b_2")
#     e_b_3 = graph.get_tensor_by_name("e_b_3")
#     d_b_1 = graph.get_tensor_by_name("d_b_1:0")
#     d_b_2 = graph.get_tensor_by_name("d_b_2:0")
#     d_b_3 = graph.get_tensor_by_name("d_b_3:0")
#     b_1 = graph.get_tensor_by_name("b_1:0")
#     b_2 = graph.get_tensor_by_name("b_2:0")
#     w_3 = tf.Variable(tf.truncated_normal([128, output], stddev = 0.1),name='w_3')
#     b_3 = tf.Variable(tf.constant(0.0, shape=[output]),name='b_3')
#     var_to_init = [w_3,b_3]
#     tf.initialize_variables(var_to_init)
    predict_output = out
    correct_prediction = tf.equal(tf.argmax(predict_output, 1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    training_epochs = 15
    batch_size = 10
    total_batches = training_data.shape[0]
    s_cost_function = -tf.reduce_sum(Y * tf.log(predict_output)) 
    
    g_vars=tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='output')
    s_optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(s_cost_function,var_list = g_vars)
    #s_optimizer = tf.train.AdamOptimizer(1e-4).minimize(s_cost_function,var_list = g_vars)
# ---------------- Training NN - Supervised Learning ------------------ #
    for epoch in range(training_epochs):
        epoch_costs = np.empty(0)
        for b in range(total_batches):
            offset = (b * batch_size) % (train_x.shape[0] - batch_size)
            batch_x = X_train_tar[offset:(offset + batch_size), :]
            batch_y = Y_train_tar[offset:(offset + batch_size), :]
            _, c = sess2.run([s_optimizer, s_cost_function],feed_dict={X: batch_x, Y : batch_y})
            epoch_costs = np.append(epoch_costs,c)
#         print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs))
#     print("------------------------------------------------------------------")

        accuracy_in_train_set = sess2.run(accuracy, feed_dict={X: X_train_tar, Y: Y_train_tar})
        accuracy_in_test_set = sess2.run(accuracy, feed_dict={X: X_test_tar, Y: Y_test_tar})
        print("Epoch: ",epoch," Loss: ",np.mean(epoch_costs)," Accuracy: ", accuracy_in_train_set, ' ', accuracy_in_test_set)

INFO:tensorflow:Restoring parameters from ./cppp1\test-model-1000
Epoch:  0  Loss:  0.375006341356  Accuracy:  0.835264   0.766467
Epoch:  1  Loss:  0.190820695405  Accuracy:  0.859717   0.772455
Epoch:  2  Loss:  0.15672078365  Accuracy:  0.885457   0.772455
Epoch:  3  Loss:  0.140091978474  Accuracy:  0.890605   0.781437
Epoch:  4  Loss:  0.128741551366  Accuracy:  0.895753   0.784431
Epoch:  5  Loss:  0.119989182841  Accuracy:  0.903475   0.784431
Epoch:  6  Loss:  0.112861313659  Accuracy:  0.912484   0.784431
Epoch:  7  Loss:  0.106869272051  Accuracy:  0.916345   0.784431
Epoch:  8  Loss:  0.101723578302  Accuracy:  0.924067   0.778443
Epoch:  9  Loss:  0.0972287182819  Accuracy:  0.927928   0.787425
Epoch:  10  Loss:  0.093251643257  Accuracy:  0.929215   0.787425
Epoch:  11  Loss:  0.0896971410499  Accuracy:  0.933076   0.793413
Epoch:  12  Loss:  0.0864894038983  Accuracy:  0.933076   0.787425
Epoch:  13  Loss:  0.0835806693159  Accuracy:  0.936937   0.784431
Epoch:  14  Loss: